# Creational Patterns

Creational patterns deal with the creation/construction of objects:

- The creation can be explicit (e.g., via constructors) of implicit (e.g., dependency injection, reflection, etc.).
- The creation can be wholesale (single statement creates the object) or piecewise, i.e., it is done step-by-step in a more complicated process.

Table of contents:

- [Builder](#Builder)
  - Example: HTML Builder
  - Car Builder
  - Person Builder
  - Product Builder with Director
- [Factories](#Factories)
  - Factory Methods
  - Factory Classes
  - Abstract Factory
- [Prototype](#Prototype)
  - Prototype Factory
- [Singleton](#Singleton)
  - Singleton Allocator
  - Singleton Decorator
  - Singleton Metaclass
  - Monostate
  - Singleton Testability

## Builder

Sometimes objects are easy to create and they are built witha single initializer call. However, in some other cases creating an object is more complicated

- it requires several steps
- or it dozens of parameters are involved
- or a piecewise construction is required.

The **Builder** provides an API for constructing an object step-by-step: when piecewise object construction is complicated, it provides an API for doing it succinctly.

Different examples are provided: an HTML builder, a Car builder and a Person builder. In general:

- We have an Object class and a Builder class which builds those objects.
- The Builder has methods that added components to the Object.
- We can either explicitly use the Builder or return it from the Object using a `create()` static method.
- We can make the builder fluent so that the methods that add components are chained.
- Different facets of an Object can be built with different Builder classes that work in tandem.
- We can define a Director class which `constructs()` different types of Objects.

### Example: HTML Builder

In [1]:
# If we want to build a simple HTML paragraph using a list
hello = 'hello'
parts = ['<p>', hello, '</p>']
print(''.join(parts))

<p>hello</p>


In [2]:
# Now I want an HTML list with 2 words in it
words = ['hello', 'world']
parts = ['<ul>']
for w in words:
    parts.append(f'  <li>{w}</li>')
parts.append('</ul>')
print('\n'.join(parts))

<ul>
  <li>hello</li>
  <li>world</li>
</ul>


We see that the complexity explodes as we start having rich HTML pages with different components. Solution, we define classes that carry out the building task together: `HtmlElement` and `HtmlBuilder`.

In [1]:
class HtmlElement:
    indent_size = 2

    def __init__(self, name="", text=""):
        self.name = name # div, p, etc.
        self.text = text
        self.elements = [] # we will populate this list

    def __str(self, indent):
        """This function recursively calls itself to produce
        the HTML page text:
        
        <{name}>
        {text}
            <{name}>
            {text}
                <{name}>
                {text}
                    ...
                </{name}>                
            </{name}>
        </{name}>
        """
        lines = []
        i = ' ' * (indent * self.indent_size)
        lines.append(f'{i}<{self.name}>')

        if self.text:
            i1 = ' ' * ((indent + 1) * self.indent_size)
            lines.append(f'{i1}{self.text}')

        for e in self.elements:
            # ATTENTION: recursive call with updated indentation
            lines.append(e.__str(indent + 1))

        lines.append(f'{i}</{self.name}>')
        return '\n'.join(lines)

    def __str__(self):
        # The goal is to print the HTML element
        # but for that we need the correct indentation.
        # To that end, we call an internal __str()
        # which takes the correct indentation
        return self.__str(0)

    # This is added at the end, as an add on.
    # In some way this is breaking the Open-Close principle
    # and it seems a circular dependency injection.
    # However, it allows to give us an alternative to just use
    # HtmlElement only, i.e., the user interacts only with
    # HtmlElement and not HtmlBuilder.
    # Keep in mind that HtmlBuilder is used behind the scenes all the time
    # but the user would have to invoke it only in the beginning.
    # Alternative solution: use a factory, shown later. 
    @staticmethod
    def create(name):
        return HtmlBuilder(name)

In [7]:
class HtmlBuilder:
    #__root = HtmlElement()

    def __init__(self, root_name):
        self.__root = HtmlElement()
        self.root_name = root_name
        self.__root.name = root_name

    # Building API: ordinary, not fluent
    def add_child(self, child_name, child_text):
        self.__root.elements.append(
            HtmlElement(child_name, child_text)
        )

    # Building API: fluent
    # A fluent API allows to chain calls one after another:
    # builder.add_child_fluent(...).add_child_fluent(...)
    def add_child_fluent(self, child_name, child_text):
        self.__root.elements.append(
            HtmlElement(child_name, child_text)
        )
        # Fluent APIs are created by returning self
        return self

    def clear(self):
        self.__root = HtmlElement(name=self.root_name)

    def __str__(self):
        return str(self.__root)

In [11]:
# Ordinary non-fluent builder
#builder = HtmlBuilder('ul')
builder = HtmlElement.create('ul')
builder.add_child('li', 'hello')
builder.add_child('li', 'world')
print('Ordinary builder:')
print(builder)

Ordinary builder:
<ul>
  <li>
    hello
  </li>
  <li>
    world
  </li>
</ul>


In [12]:
# Fluent builder
builder.clear()
(builder
    .add_child_fluent('li', 'hello')
    .add_child_fluent('li', 'world')
)
print('Fluent builder:')
print(builder)

Fluent builder:
<ul>
  <li>
    hello
  </li>
  <li>
    world
  </li>
</ul>


In [16]:
# This alternative would allow us to remove the static method create()
# from HtmlElement, which solves breaking the Open-Close principle.
class HtmlElementFactory:
    @staticmethod
    def create(name):
        return HtmlBuilder(name)

# Usage
builder = HtmlElementFactory.create('ul')
builder.add_child_fluent('li', 'hello').add_child_fluent('li', 'world')
print(builder)

<ul>
  <li>
    hello
  </li>
  <li>
    world
  </li>
</ul>


### Example: Car Builder

In [14]:
class Car:
    def __init__(self):
        self.features = []

    def add_feature(self, feature):
        self.features.append(feature)

    def __str__(self):
        return ', '.join(self.features)

class CarBuilder:
    def __init__(self):
        self.car = Car()

    def add_engine(self, engine):
        self.car.add_feature(f"Engine: {engine}")
        return self

    def add_wheels(self, wheels):
        self.car.add_feature(f"Wheels: {wheels}")
        return self

    def add_gps(self):
        self.car.add_feature("GPS: Installed")
        return self

    def build(self):
        return self.car

class Director:
    def construct_sports_car(self, builder):
        return builder.add_engine('V8').add_wheels('Sports').add_gps().build()

    def construct_family_car(self, builder):
        return builder.add_engine('V4').add_wheels('Regular').add_gps().build()

# Usage
director = Director()
sports_car_builder = CarBuilder()
family_car_builder = CarBuilder()

sports_car = director.construct_sports_car(sports_car_builder)
family_car = director.construct_family_car(family_car_builder)

print(sports_car)  # Engine: V8, Wheels: Sports, GPS: Installed
print(family_car)  # Engine: V4, Wheels: Regular, GPS: Installed


Engine: V8, Wheels: Sports, GPS: Installed
Engine: V4, Wheels: Regular, GPS: Installed


### Example: Person Builder

In [2]:
class Person:
    def __init__(self):
        print('Creating an instance of Person')
        # address
        self.street_address = None
        self.postcode = None
        self.city = None
        # employment info
        self.company_name = None
        self.position = None
        self.annual_income = None

    def __str__(self) -> str:
        return f'Address: {self.street_address}, {self.postcode}, {self.city}\n' +\
            f'Employed at {self.company_name} as a {self.position} earning {self.annual_income}'

In [3]:
class PersonBuilder:  # facade
    def __init__(self, person=None):
        if person is None:
            self.person = Person()
        else:
            self.person = person

    def build(self):
        return self.person
    
    # It seems circular, but it's not:
    # PersonAddressBuilder and PersonJobBuilder
    # extend PersonBuilder by inheritance.
    # They all depend on Person, which is passed
    # through the constructor.
    # However, it is violating the Open-Close principle!
    # We could solve it with inheritance; see builder_inheritance.py
    @property
    def lives(self):
        return PersonAddressBuilder(self.person)

    @property
    def works(self):
        return PersonJobBuilder(self.person)

In [4]:
class PersonJobBuilder(PersonBuilder):
    def __init__(self, person):
        super().__init__(person)

    def at(self, company_name):
        self.person.company_name = company_name
        return self

    def as_a(self, position):
        self.person.position = position
        return self

    def earning(self, annual_income):
        self.person.annual_income = annual_income
        return self

In [5]:
class PersonAddressBuilder(PersonBuilder):
    def __init__(self, person):
        super().__init__(person)

    def at(self, street_address):
        self.person.street_address = street_address
        return self

    def with_postcode(self, postcode):
        self.person.postcode = postcode
        return self

    def in_city(self, city):
        self.person.city = city
        return self

In [6]:
# Usage
pb = PersonBuilder()
p = pb\
    .lives\
        .at('123 London Road')\
        .in_city('London')\
        .with_postcode('SW12BC')\
    .works\
        .at('Fabrikam')\
        .as_a('Engineer')\
        .earning(123000)\
    .build()
print(p)
person2 = PersonBuilder().build()
print(person2)

Creating an instance of Person
Address: 123 London Road, SW12BC, London
Employed at Fabrikam as a Engineer earning 123000
Creating an instance of Person
Address: None, None, None
Employed at None as a None earning None


### Product Builder with Director

Another example, taken from [All 23 OOP software design patterns with examples in Python](https://medium.com/@cautaerts/all-23-oop-software-design-patterns-with-examples-in-python-cac1d3f4f4d5):

- **Product**: This is the complex object under construction. It contains the parts that make up the product.

- **Builder**: The Builder defines the abstract interface for creating parts of the Product object. It declares the construction steps that are common to all types of builders. This allows the Director to construct products using any builder that implements this interface, without knowing the specific details of the construction process.

- **ConcreteBuilder**: The ConcreteBuilder implements the Builder interface and provides specific implementations for the construction steps. It keeps track of the product it creates and is responsible for assembling the parts into the final product. It knows the specifics of the parts it is supposed to create and how to assemble them.

- **Director**: The Director is responsible for managing the construction process. It uses a Builder instance to construct a product step-by-step. The Director doesn’t need to know the details of the product's construction, it only needs to ensure that the construction steps are followed in the correct order.

In [7]:
from abc import ABC, abstractmethod
from typing import List

class Product:
   def __init__(self) -> None:
       self._parts: List[str] = []

   def add_part(self, part: str) -> None:
       self._parts.append(part)

   def get_parts(self) -> List[str]:
       return self._parts

class Builder(ABC):
    @abstractmethod
    def build_part_a(self) -> None:
        pass

    @abstractmethod
    def build_part_b(self) -> None:
        pass

    @abstractmethod
    def get_result(self) -> Product:
        pass

class ConcreteBuilder(Builder):
    def __init__(self) -> None:
        self.product = Product()

    def build_part_a(self) -> None:
        self.product.add_part("Part A")

    def build_part_b(self) -> None:
        self.product.add_part("Part B")

    def get_result(self) -> Product:
        return self.product

class Director:
    def __init__(self, builder: Builder) -> None:
        self._builder = builder

    def construct(self) -> None:
        self._builder.build_part_a()
        self._builder.build_part_b()

# Example usage
builder = ConcreteBuilder()
director = Director(builder)
director.construct()
product = builder.get_result()

print(product.get_parts())  # Output: ['Part A', 'Part B']


['Part A', 'Part B']


## Factories

A Factory is a component responsible solely for the wholesale (not piecewise) creation of objects.

- Sometimes object creation logic becomes too convoluted.
- Class initialization with `__init__()` is indeed a bit limited:
  - Name is always `__init__()`.
  - Cannot be overloaded with same sets of arguments with different names
  - Can turn into an optional parameter hell.
- Solution: Factories - wholesale object creation (non-piecewise, i.e., not defined in steps, unlike Builder) that can be outsourced to
  - a set separate methods: **factory methods**;
  - that can may exist in a separate class **Factory**, which contains the factory methods;
  - And we can create a hierarchy of factories, i.e., **Abstract Factories**, which are factory classes created for abstract and derived object classes.

In the following, two examples are shown to illustrate the abovementioned 3 concepts:

- Point class: Factory Methods & Classes
- Hot drink class: Abstract Factory

### Factory Methods

Instead of having massive constructors which take large amounts of parameters, the idea is to have smaller static methods that take subsets of those parameters and use them to build internally the object, which usually requires a minor subset of all parameters to represent the object state.

In [8]:
# Let's consider a Point, which can be expressed in Cartesian
# or polar coordinates. Unfortunately though,
# we can only define a unique __init__ constructor
class Point:
    def __init__(self, x, y):
        self.x = x
        self.y = y

In [16]:
from enum import Enum
import math

# One quick and dirty implementation would be to pass
# a system argument and treat the other arguments
# differently.
# BUT that's an issue, because it complicates the usage
# and everything is more confusing
class CoordinateSystem(Enum):
    CARTESIAN = 1
    POLAR = 2
    
class Point:
    def __init__(self, a, b, system=CoordinateSystem.CARTESIAN):
        if system == CoordinateSystem.CARTESIAN:
            self.x = a
            self.y = b
        elif system == CoordinateSystem.POLAR:
            self.x = a * math.sin(b)
            self.y = a * math.cos(b)

In [17]:
# Instead of doing the above,
# we create factory methods inside (static methods)
# which deal with the correct coordinate system
# and return an object of the class itself!
class Point:

    # We take the previous massive/extended constructor
    # but we could actually simplify it to the first version!
    def __init__(self, a, b, system=CoordinateSystem.CARTESIAN):
        if system == CoordinateSystem.CARTESIAN:
            self.x = a
            self.y = b
        elif system == CoordinateSystem.POLAR:
            self.x = a * math.sin(b)
            self.y = a * math.cos(b)
    
    # Factory method 1
    @staticmethod
    def new_cartesian_point(x, y):
        return Point(x, y)

    # Factory method 2
    @staticmethod
    def new_polar_point(rho, theta):
        return Point(rho * math.sin(theta), rho * math.cos(theta))

    def __str__(self):
        return f'x: {self.x}, y: {self.y}'


In [18]:
p1 = Point(2, 3, CoordinateSystem.CARTESIAN) # Extended constructor
p2 = Point.new_cartesian_point(1, 2) # Factory Method 1
p2 = Point.new_polar_point(3, 0.707) # Factory Method 2
print(p1)
print(p2)

x: 2, y: 3
x: 1.9486672666692673, y: 2.280941885233319


### Factory Class

When we start having many factory methods it makes sense to build a class with all of them; that way, everything is more organized and clear. We can even nest that class internally to the object class so that it is clear where it belongs.

In [19]:
# When we start having many factory methods
# it makes sense to build a class with all of them;
# that way, everything is more organized and clear
class PointFactory:
    @staticmethod
    def new_cartesian_point(x, y):
        return Point(x, y)

    @staticmethod
    def new_polar_point(rho, theta):
        return Point(rho * math.sin(theta), rho * math.cos(theta))

In [21]:
p3 = PointFactory.new_polar_point(1, 0.5) # Factory Class -> Factory Method
print(p3)

x: 0.479425538604203, y: 0.8775825618903728


In [28]:
# Since the Factory class is separate from the Object class
# if we want to make sure that it is not lost
# we can even include it into the object class itself
class Point:
    def __init__(self, x, y):
       self.x = x
       self.y = y

    def __str__(self):
        return f'x: {self.x}, y: {self.y}'

    class Factory:
        @staticmethod
        def new_cartesian_point(x, y):
            return Point(x, y)

        @staticmethod
        def new_polar_point(rho, theta):
            return Point(rho * math.sin(theta), rho * math.cos(theta))

In [29]:
p4 = Point.Factory.new_polar_point(1, 0.5) # Object -> Factory Class -> Factory Method
print(p4)

x: 0.479425538604203, y: 0.8775825618903728


### Abstract Factory

Idea: if you have a hierarchy of types, then you can have also a hierarchy of factories.

In [35]:
from abc import ABC, abstractmethod
from typing import Type, Dict

# Abstract Object
class HotDrink(ABC):
    @abstractmethod
    def consume(self):
        pass

# Concrete Object 1
class Tea(HotDrink):
    def consume(self):
        print("This tea is nice but I'd prefer it with milk")

# Concrete Object 2
class Coffee(HotDrink):
    def consume(self):
        print('This coffee is delicious')

# Abstract Object Factory
class HotDrinkFactory(ABC):
    @abstractmethod
    def prepare(self, amount: int) -> HotDrink:
        pass

# Concrete Object 1 Factory
class TeaFactory(HotDrinkFactory):
    def prepare(self, amount: int) -> Tea:
        print(f'Put in tea bag, boil water, pour {amount}ml, enjoy!')
        return Tea()

# Concrete Object 2 Factory
class CoffeeFactory(HotDrinkFactory):
    def prepare(self, amount: int) -> Coffee:
        print(f'Grind some beans, boil water, pour {amount}ml, enjoy!')
        return Coffee()

In [36]:
# Then, we need a function that uses all of them!
# We could even create a class HotDrinkMachine
# which has make_drink() as method, as done below
def make_drink(type):
    if type == 'tea':
        return TeaFactory().prepare(200)
    elif type == 'coffee':
        return CoffeeFactory().prepare(50)
    else:
        return None

In [41]:
# Usage
drink = make_drink("tea")
drink.consume()

Put in tea bag, boil water, pour 200ml, enjoy!
This tea is nice but I'd prefer it with milk


In [38]:
# HotDrinkMachine class, which contains make_drink()
# Additionally, we add the functionality of adding new Factories!
class HotDrinkMachine:
    def __init__(self):
        self.factories: Dict[str, Type[HotDrinkFactory]] = {
            'tea': TeaFactory,
            'coffee': CoffeeFactory
        }
    
    def add_factory(self, drink_type: str, factory: Type[HotDrinkFactory]) -> None:
        self.factories[drink_type] = factory
    
    def make_drink(self, drink_type: str, amount: int) -> HotDrink:
        factory = self.factories.get(drink_type)
        if not factory:
            raise ValueError(f"No factory found for drink type: {drink_type}")
        return factory().prepare(amount)

In [39]:
# Example usage with pre-defined factories
machine = HotDrinkMachine()
tea = machine.make_drink('tea', 200)
tea.consume()

coffee = machine.make_drink('coffee', 50)
coffee.consume()

Put in tea bag, boil water, pour 200ml, enjoy!
This tea is nice but I'd prefer it with milk
Grind some beans, boil water, pour 50ml, enjoy!
This coffee is delicious


In [40]:
# Now, we create a new Factory and add it!
# With our HotDrinkMachine implementation we have achieved
# adding a new drink type without modifying existing code,
# i.e., we avoid breaking the Open-Close principle
class HotChocolate(HotDrink):
    def consume(self):
        print('This hot chocolate is delightful!')

class HotChocolateFactory(HotDrinkFactory):
    def prepare(self, amount: int) -> HotChocolate:
        print(f'Mix chocolate powder with {amount}ml of hot water, enjoy!')
        return HotChocolate()

# Adding the new factory to the machine
machine.add_factory('hot_chocolate', HotChocolateFactory)
hot_chocolate = machine.make_drink('hot_chocolate', 250)
hot_chocolate.consume()

Mix chocolate powder with 250ml of hot water, enjoy!
This hot chocolate is delightful!


## Prototype

Prototype: We use them when it's easier to copy an existing object to fully initialize a new one. Thus, a Prototype is a partially or fully initialized object that we copy (*clone*) and make use of it:

- Complicated objects (e.g., cars) aren't designed from scratch.
  - They reiterate existing designs.
- An existing (partially or fully constructed) design is a Prototype.
- We want to copy it (*clone*), customize it, and start using it.
  - It requires a *deep copy* support, i.e., a wholesale copy.
  - Maybe the entire object is not created in the Prototype, but some parts only.
- We make the *cloning* convenient: e.g., via a **Factory** (**Prototype Factory**):
  - We define the Object class we'd like to clone.
  - We deine a Factory class for it.
  - In the Factory class, we partially construct some object templates.
  - For each object template, we create static factory methods.
  - Each static factory method clones == deep copies a template and customizes it.
  - We have a nice and easy API for the users that creates complex objects with few lines of code.

In [46]:
import copy

# Let's take the example of classes
# for which we would like to have already
# pre-conigured object instances
class Address:
    def __init__(self, street_address, city, country):
        self.country = country
        self.city = city
        self.street_address = street_address

    def __str__(self):
        return f'{self.street_address}, {self.city}, {self.country}'
    
class Employee:
    def __init__(self, name, address):
        self.address = address
        self.name = name

    def __str__(self):
        return f'{self.name} works at {self.address}'

In [47]:
john = Employee("John", Address("123 London Road", "London", "UK"))
print(john)
# jane = john # this is not really a copy, because Employee is mutable...
jane = copy.deepcopy(john) # Real copy
jane.name = "Jane"
jane.address.street_address = "124 London Road"
print(john, jane)

John works at 123 London Road, London, UK
John works at 123 London Road, London, UK Jane works at 124 London Road, London, UK


In [48]:
# To build from prototypes, we define a factory
# which contains pre-configured templates of an object
class EmployeeFactory:
    # These are the prototypes which belong to the class,
    # i.e., the same for all class objects
    # since they are not defined in self
    main_office_employee = Employee("", Address("123 East Dr", 0, "London"))
    aux_office_employee = Employee("", Address("123B East Dr", 0, "London"))

    @staticmethod
    def __new_employee(proto, name, suite):
        result = copy.deepcopy(proto)
        result.name = name
        result.address.suite = suite
        return result

    @staticmethod
    def new_main_office_employee(name, suite):
        return EmployeeFactory.__new_employee(
            EmployeeFactory.main_office_employee,
            name, suite
        )

    @staticmethod
    def new_aux_office_employee(name, suite):
        return EmployeeFactory.__new_employee(
            EmployeeFactory.aux_office_employee,
            name, suite
        )

In [51]:
john = EmployeeFactory.new_aux_office_employee("John", 200)
jane = EmployeeFactory.new_main_office_employee("Jane", 200)
print(jane)
print(john)

Jane works at 123 East Dr, 0, London
John works at 123B East Dr, 0, London


## Singleton

The Gang of Four have expressed that if they had to drop a design pattern, that would be the *Singleton*, because it is often times a design smell.

Singletons are useful for system components that usually should have one instance which should be initialized only once; so the goal of the Singleton is to prevent the user from initializing multiple instances:

- Database repositories
- Object factories
- etc.

Why do we need only one instance?

- Initializers are expensive
  - We only do it once
  - We provide everyone with the same instance
- We want to prevent anyone creating additional copies
- We need to take care of lazy instantiation: Usually, we initialize the Singleton whe it is needed, i.e., we don't initialize it preventitively.